In [1]:
import pandas as pd
import xgboost
import datetime; import pytz
import matplotlib as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder

In [2]:

def utc_ts():
    return datetime.datetime.utcnow(
        ).replace(tzinfo=pytz.UTC).strftime('%Y-%m-%dT%H%M%SZ')
utc_ts()

'2020-06-08T142456Z'

#### The data


In [3]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [15]:
def preprocess(df, stationsdf, labeled):
    # Initially assuming labeled=True
    labeled = True
    
    # Step 1, merge w/ stationsdf to get neighborhood data
    mdf = tripsdf[['start station name', 'end station name', 'gender']
            ].merge(stationsdf[['station_name', 'neighborhood']], 
                    left_on='start station name',
                    right_on='station_name'
                   ).rename(columns={'neighborhood': 'start_neighborhood'}
                           ).merge(stationsdf[['station_name', 'neighborhood']],
                                  left_on='end station name',
                                   right_on='station_name'
                                  ).rename(columns={'neighborhood': 'end_neighborhood'})
    
    neighborhoods = sorted(stationsdf.neighborhood.unique().tolist())
    genders = [0, 1, 2]
    
    X, y = (mdf[['start_neighborhood', 'gender']].values, 
            np.array(mdf['end_neighborhood'].tolist()))
    
    enc = OneHotEncoder(handle_unknown='error', 
                        categories=[neighborhoods, genders])
    enc.fit(X)
    X_transformed = enc.transform(X)
    return X, X_transformed, y, enc
    
    

In [16]:
tripsdf.iloc[0]

tripduration                               634
starttime                  2013-07-01 00:00:00
stoptime                   2013-07-01 00:10:34
start station id                           164
start station name             E 47 St & 2 Ave
start station latitude                 40.7532
start station longitude               -73.9703
end station id                             504
end station name               1 Ave & E 15 St
end station latitude                   40.7322
end station longitude                 -73.9817
bikeid                                   16950
usertype                              Customer
birth year                                  \N
gender                                       0
Name: 0, dtype: object

In [17]:
(X, X_transformed, y, enc) = preprocess(tripsdf, stationsdf, labeled=True)

In [20]:
X_transformed[0].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [8]:
# Get data..
xgboost.DMatrix()

xgboost.core.DMatrix

In [18]:
neighborhoods = ['Midtown East', 'Stuyvesant Town',]; gender = [0,1,2]
enc = OneHotEncoder(handle_unknown='ignore', categories=[neighborhoods, gender])
X = np.array([['Midtown East', 1],
             ['Midtown East',2],
             ['Stuyvesant Town',0]])
X2 = np.array([['foo', 1],
             ['Midtown East',2],
             ['Stuyvesant Town',0]])
enc.fit(X)


OneHotEncoder(categories=[['Midtown East', 'Stuyvesant Town'], [0, 1, 2]],
              drop=None, dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              sparse=True)

In [19]:
enc.transform(X2).toarray()

array([[0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.]])